In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

spark = SparkSession.builder \
    .appName('NSL-KDD') \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

train_data = spark.read.csv('E:/Fall 24/Systems and Tools/NSL-KDD/KDDTrain+.txt', header=False, inferSchema=True)
test_data = spark.read.csv('E:/Fall 24/Systems and Tools/NSL-KDD/KDDTest+.txt', header=False, inferSchema=True)

train_data = train_data.withColumn("data_type", lit("train"))
test_data = test_data.withColumn("data_type", lit("test"))

combined_data = train_data.union(test_data)

url = "jdbc:postgresql://localhost:5432/postgres"  
properties = {
    "user": "postgres",
    "password": "Sakhaleyash@1",
    "driver": "org.postgresql.Driver"
}

combined_data.write.jdbc(url=url, table="nsl_kdd_data", mode="overwrite", properties=properties)
sample_data = spark.read.jdbc(url=url, table="nsl_kdd_data", properties=properties)

# Verifying the data_type column has been added properly, I have only displayed random columns because displaying the full dataset is too wide for the output window
print("Showing a sample of the combined dataset including the 'data_type' column:")
combined_data.select("_c0", "_c1","_c2","_c3","_c4","_c5","_c8","_c17","_c23","_c26","_c31","_c35", "_c41", "data_type").show(8, truncate=False)


Showing a sample of the combined dataset including the 'data_type' column:
+---+---+--------+---+---+----+---+----+----+----+----+----+-------+---------+
|_c0|_c1|_c2     |_c3|_c4|_c5 |_c8|_c17|_c23|_c26|_c31|_c35|_c41   |data_type|
+---+---+--------+---+---+----+---+----+----+----+----+----+-------+---------+
|0  |tcp|ftp_data|SF |491|0   |0  |0   |2   |0.0 |150 |0.17|normal |train    |
|0  |udp|other   |SF |146|0   |0  |0   |1   |0.0 |255 |0.88|normal |train    |
|0  |tcp|private |S0 |0  |0   |0  |0   |6   |0.0 |255 |0.0 |neptune|train    |
|0  |tcp|http    |SF |232|8153|0  |0   |5   |0.0 |30  |0.03|normal |train    |
|0  |tcp|http    |SF |199|420 |0  |0   |32  |0.0 |255 |0.0 |normal |train    |
|0  |tcp|private |REJ|0  |0   |0  |0   |19  |1.0 |255 |0.0 |neptune|train    |
|0  |tcp|private |S0 |0  |0   |0  |0   |9   |0.0 |255 |0.0 |neptune|train    |
|0  |tcp|private |S0 |0  |0   |0  |0   |16  |0.0 |255 |0.0 |neptune|train    |
+---+---+--------+---+---+----+---+----+----+----+----+-